<link rel="stylesheet" href="../../styles/theme_style.css">
<!--link rel="stylesheet" href="../../styles/header_style.css"-->
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">

<table width="100%">
    <tr>
        <td id="image_td" width="15%" class="header_image_color_3"><div id="image_img" class="header_image_3"></div></td>
        <!-- Available classes for "image_td" element:
        - header_image_color_1 (For Notebooks of "Open" Area);
        - header_image_color_2 (For Notebooks of "Acquire" Area);
        - header_image_color_3 (For Notebooks of "Visualise" Area);
        - header_image_color_4 (For Notebooks of "Process" Area);
        - header_image_color_5 (For Notebooks of "Detect" Area);
        - header_image_color_6 (For Notebooks of "Extract" Area);
        - header_image_color_7 (For Notebooks of "Decide" Area);
        - header_image_color_8 (For Notebooks of "Explain" Area);

        Available classes for "image_img" element:
        - header_image_1 (For Notebooks of "Open" Area);
        - header_image_2 (For Notebooks of "Acquire" Area);
        - header_image_3 (For Notebooks of "Visualise" Area);
        - header_image_4 (For Notebooks of "Process" Area);
        - header_image_5 (For Notebooks of "Detect" Area);
        - header_image_6 (For Notebooks of "Extract" Area);
        - header_image_7 (For Notebooks of "Decide" Area);
        - header_image_8 (For Notebooks of "Explain" Area);-->
        <td class="header_text"> Plotting of Acquired Data using Bokeh </td>
    </tr>
</table>

<div id="flex-container">
    <div id="diff_level" class="flex-item">
        **Difficulty Level:**   <span class="fa fa-star checked"></span>
                                <span class="fa fa-star checked"></span>
                                <span class="fa fa-star"></span>
                                <span class="fa fa-star"></span>
                                <span class="fa fa-star"></span>
    </div>
    <div id="tag" class="flex-item-tag">
        <span id="tag_list">
            <table id="tag_list_table">
                <tr>
                    <td class="shield_left">Tags</td>
                    <td class="shield_right" id="tags">visualise|plot|single</td> 
                </tr>
            </table>
        </span>
        <!-- [OR] Visit https://img.shields.io in order to create a tag badge-->
    </div>
</div>

Digital sensors, like the ones used in *Plux* acquisition systems, establish an interface between physical and digital environment.

The human sensorial system is analogous to this description, being the visual component particularly important for a researcher, in order to extract knowledge from the acquired data. Plotting data brings another perspective to the researcher, stimulating this sensorial component, which is ideal for identifying patterns and communicate.

This **<span class="color5">Jupyter Notebook</span>** is intended to explain how the user can plot data in a simple and attractive way.

<hr>

<p class="steps">1 - Importation of the needed packages</p>

In [1]:
# Base packages used in OpenSignals Tools Notebooks for ploting data
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.layouts import gridplot
from bokeh.models.tools import *
output_notebook(hide_banner=True)

# Package for ensuring operations in arrays
import numpy

# Own opensignalstools package needed here for applying the default OpenSignals styles and for loading data from an acquisition 
# file.
import opensignalstools.open as ostOpen
import opensignalstools.visualise as ostVis

<p class="steps">2 - Load of data using <span class="color1">loadData</span> function of **<span class="color2">opensignalstools</span>** package</p>

In [2]:
# Remote URL where signals are stored
biosignals_url = "http://www.biosignalsplux.com/downloads/samples/sensor_samples"
filename = "biosignalsplux_Electrocardiogram_(ECG)_Sample.txt"

# Load of data from a remote source
print ("Downloading...\n")
data, header = ostOpen.loadData(file=biosignals_url + "/" + filename, getHeader=True, remote=True)

print ("\n\nHeader:")
print (header)
print ("\nData")
print (data)

Downloading...

100% [..............................................................................] 34923 / 34923

Header:
{'00:07:80:3B:46:61': {'device name': '00:07:80:3B:46:61', 'sync interval': 2, 'time': '14:50:32.316', 'comments': '', 'device connection': 'BTH00:07:80:3B:46:61', 'channels': [1], 'date': '2017-1-17', 'digital IO': [0, 1], 'firmware version': 772, 'device': 'biosignalsplux', 'sampling rate': 200, 'resolution': [16], 'column labels': {1: 2}}}

Data
{'00:07:80:3B:46:61': {'CH1': array([ 32452.,  32394.,  32448., ...,  33120.,  33164.,  33192.])}}


<p class="steps">3 - Creation of a time axis for the acquisitions of each device</p>

In [3]:
# Acquisition metadata
macs = header.keys()

# Inclusion of additional fields to data dictionary
for mac in macs:
    sampling_freq = header[mac]["sampling rate"]
    for chn in data[mac].keys():
        # Channel data
        chn_data = data[mac][chn]
        
        # Number of channel samples
        nbr_samples = len(chn_data)
        
        # Acquisition time (last instant)
        last_instant = nbr_samples / sampling_freq
        
        # Generation and storage of the time axis
        chn_time = numpy.linspace(0, last_instant, nbr_samples)
        data[mac][chn] = {"data": chn_data, "time": chn_time}

<p class="steps">4 - Creation of an auxiliary signal through adding noise to the original one</p>

In [4]:
# Noise samples and change of the baseline level
for mac in macs:
    for chn in data[mac].keys():
        baseline = numpy.average(data[mac][chn]["data"])
        baseline_shift = 0.50 * baseline
        data[mac][chn]["noise"] = data[mac][chn]["data"] + numpy.random.normal(0, 1000, nbr_samples) + baseline_shift

<p class="steps">5 - Creation of a list intended to store the figure handlers (needed for future application of **<span class="color2">OpenSignals</span>** style)</p>

In [5]:
# Creation of a Bokeh figure list.
figure_list = []

<p class="steps">6 - Creation of a figure to plot the acquired data</p> *It is not necessary to store figures in a list, like is done here. But for the present application this approach is used to ensure that the <span class="color2">OpenSignals</span> style is applied to all the generated figures*</p>

In [6]:
figure_list.append(figure(x_axis_label='Time (s)', y_axis_label='Raw Data', **ostVis.opensignalsKwargs("figure")))

<p class="steps">7 - Plotting of data in the last created figure (adding the respective legend of the axes with the "legend" argument)</p>

In [7]:
for mac in macs:
    for chn in data[mac].keys():
        # Original Data.
        figure_list[-1].line(data[mac][chn]["time"], data[mac][chn]["data"], legend="Original Data", **ostVis.opensignalsKwargs("line"))
        
        # Noisy Data.
        figure_list[-1].line(data[mac][chn]["time"], data[mac][chn]["noise"], legend="Noisy Data", **ostVis.opensignalsKwargs("line"))

*Bokeh offers lots of configurable options. When invoking line function arguments can be specified for customizing the plot, such as the next ones:*
<table align="left">
    <tr style="border-bottom: solid 1px">
        <td style="text-align:center;vertical-align:middle">**Argument**</td>
        <td style="text-align:left;vertical-align:middle">**Input type**</td>
    </tr>
    <tr>
        <td style="text-align:center;vertical-align:middle">*line_width*</td>
        <td style="text-align:left;vertical-align:middle">numeric</td>
    </tr>
    <tr>
        <td style="text-align:center;vertical-align:middle">*line_color*</td>
        <td style="text-align:left;vertical-align:middle">*RGB hexadecimal values; <br>bokeh.colors.RGB objects;<br>CSS-format RGB/RGBA strings <br>*</td>
    </tr>
    <tr>
        <td style="text-align:center;vertical-align:middle">*line_dash*</td>
        <td style="text-align:left;vertical-align:middle">*line style such as dotted or dashed*</td>
    </tr>
</table>

<p class="steps">8 - Show figure</p>
*Bokeh offers the user interactive tools in right panel, for panning or zooming the representation*

In [18]:
show(figure_list[-1])

Sometimes may be interesting that plots be represented in separate axes. This can be achieved by creating multiple figures.
When a figure is created his handler needs to be stored in a variable, in order to plot all the desired data by invoking the figure.

When all figures have been "filled" a Grid Layout is defined, as described in the following steps.

<hr>

<p class="steps">E0 - Creation of a list intended to store the figure handlers (needed for future application of **<span class="color2">OpenSignals</span>** style)</p>

In [9]:
# Creation of a Bokeh figure list.
grid_figure_list = []

<p class="steps">E.1 - Creation of two separate figures</p>

In [10]:
# Top Figure.
top_figure = figure(x_axis_label='Time (s)', y_axis_label='Raw Data', **ostVis.opensignalsKwargs("figure"))

# Bottom Figure.
bottom_figure = figure(x_axis_label='Time (s)', y_axis_label='Raw Data', **ostVis.opensignalsKwargs("figure"))

# Storage of figures in our global list.
grid_figure_list.append(top_figure)
grid_figure_list.append(bottom_figure)

<p class="steps">E.2 - Plotting of data in each figure</p>

In [11]:
# Plot of data in top_figure.
top_figure.line(data[mac][chn]["time"], data[mac][chn]["data"], legend="Original Data", **ostVis.opensignalsKwargs("line"))

# Plot of data in bottom_figure.
bottom_figure.line(data[mac][chn]["time"], data[mac][chn]["noise"], legend="Noisy Data", **ostVis.opensignalsKwargs("line"))

GlyphRenderer(id='dc28396d-c16c-402d-98f3-4a085c59eb5d', ...)

<p class="steps">E.3 - Generation of the Grid Plot structure</p>
*In this case we want a grid with 2x1 shape (2 lines and 1 column), so the gridplot input will be [[top_figure], [bottom_figure]].  
The generic strucutre of the input for NxM shape is [[<line_of_figures_1>], [<line_of_figures_2>], ... [<line_of_figures_i], ... [<line_of_figures_N>]], where [<line_of_figures_i>] is a list of M figures [<column_of_figures_1>, <column_of_figures_2>, ... <column_of_figures_M>]*

In [19]:
grid_plot = gridplot([[top_figure], [bottom_figure]], **ostVis.opensignalsKwargs("gridplot"))

<p class="steps">E.3 - Show the final Grid Plot</p>

In [20]:
show(grid_plot)

<i>Some of this visualisation functionalities can be done in a more immediate wat by **plotSimpleBokeh** function of **<span class="color2">opensignalstools</span>** package.  
For example to produce a similar result of steps 1 to 8 the command should be:  
</i>

In [14]:
ostVis.plotSimpleBokeh(time=[list(data[mac][chn]["time"]), list(data[mac][chn]["time"])], data=[list(data[mac][chn]["data"]), list(data[mac][chn]["noise"])], legend=["Original Data", "Noisy Data"], yAxisLabel=["Raw Data", "Raw Data"], x_axis_label="Time (s)")

[Figure(id='66184fd6-9869-4c31-a778-ead6d4ecfa44', ...)]

To produce the result of steps E.1 to E.3:

In [15]:
ostVis.plotSimpleBokeh(time=[list(data[mac][chn]["time"]), list(data[mac][chn]["time"])], data=[list(data[mac][chn]["data"]), list(data[mac][chn]["noise"])], legend=["Original Data", "Noisy Data"], yAxisLabel=["Raw Data", "Raw Data"], gridPlot=True, gridLines=2, gridColumns=1, x_axis_label="Time (s)")

[Figure(id='b5691c32-62cc-40c0-a064-2518200bad8d', ...),
 Figure(id='2e8c2f03-dbff-4d15-8005-13ceec3576d4', ...)]

<span class="color6">**Auxiliary Code Segment (should not be replicated by the user)**</span>

In [16]:
from IPython.display import Javascript
ostVis.opensignalsStyle(figure_list)
ostVis.opensignalsStyle(grid_figure_list)
Javascript("Jupyter.notebook.execute_cells([20, 30, 32])")

<IPython.core.display.Javascript object>

In [17]:
from opensignalstools.__notebook_support__ import cssStyleApply
cssStyleApply()

.................... CSS Style Applied to Jupyter Notebook .........................
